In [11]:
import yfinance as yf
import pandas as pd

# Create the Ticker object for Tesla
tesla = yf.Ticker("TSLA")

# Get historical stock data (maximum available)
tesla_data = tesla.history(period="max")

# Reset index so 'Date' becomes a column
tesla_data.reset_index(inplace=True)

# Display the first 5 rows
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [12]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
headers = {"User-Agent": "Mozilla/5.0"}

# Ignore SSL verification
response = requests.get(url, headers=headers, verify=False)

soup = BeautifulSoup(response.text, "html.parser")
tables = soup.find_all("table")

# Extract Tesla Revenue Table
for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        df = pd.read_html(str(table))[0]
        break

# Clean and show
df.columns = ["Date", "Revenue"]
df = df[df["Revenue"] != ""]
tesla_revenue = df

tesla_revenue.tail()

,Date,Revenue
59,2010-06-30,$28
60,2010-03-31,$21
61,2009-12-31,NaN
62,2009-09-30,$46
63,2009-06-30,$27


In [10]:
import yfinance as yf

# Download GameStop stock data
gme = yf.Ticker("GME")

# Get historical market data
gme_data = gme.history(period="max")

# Reset the index
gme_data.reset_index(inplace=True)

# Display the first 5 rows
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578048,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [21]:
import pandas as pd
import requests

# Set headers to avoid 403 error
headers = {
    "User-Agent": "Mozilla/5.0"
}

url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

# Step 1: Get HTML content
response = requests.get(url, headers=headers)
html_data = response.text

# Step 2: Read all tables
tables = pd.read_html(html_data)

# Step 3: Find the correct table by structure — [Date, Revenue]
for table in tables:
    if table.shape[1] == 2 and "Revenue" in table.columns[1]:
        gme_revenue = table
        break
else:
    raise Exception("❌ Still couldn't find GameStop revenue table.")

# Step 4: Clean and format
gme_revenue.columns = ["Date", "Revenue"]
gme_revenue["Revenue"] = gme_revenue["Revenue"].replace("[$,]", "", regex=True)
gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]
gme_revenue["Revenue"] = gme_revenue["Revenue"].astype(float)
gme_revenue["Date"] = pd.to_datetime(gme_revenue["Date"])

# Step 5: Show last 5 rows as required
print(gme_revenue.tail())

                            Date  Revenue
12 1970-01-01 00:00:00.000002013   8887.0
13 1970-01-01 00:00:00.000002012   9551.0
14 1970-01-01 00:00:00.000002011   9474.0
15 1970-01-01 00:00:00.000002010   9078.0
16 1970-01-01 00:00:00.000002009   8806.0


In [28]:
import plotly.graph_objects as go

def make_graph(stock_data, revenue_data, stock_name):
    # Ensure datetime columns are clean
    stock_data["Date"] = stock_data["Date"].dt.tz_localize(None)
    revenue_data["Date"] = pd.to_datetime(revenue_data["Date"])

    fig = go.Figure()

    # Add stock price trace
    fig.add_trace(go.Scatter(x=stock_data["Date"],y=stock_data["Close"],name="Stock Price"))

    # Add revenue trace
    fig.add_trace(go.Scatter(x=revenue_data["Date"],y=revenue_data["Revenue"],name="Revenue"))

    fig.update_layout(
        title=f"{stock_name} Stock Price and Revenue Over Time",
        xaxis_title="Date",yaxis_title="USD",legend_title="Metric",template="plotly_dark",width=800,height=450)
    fig.show()
make_graph(tesla_data, tesla_revenue, "Tesla")

In [30]:
import plotly.graph_objects as go

def make_graph(stock_data, revenue_data, stock_name):
    # Clean date columns
    stock_data["Date"] = stock_data["Date"].dt.tz_localize(None)
    revenue_data["Date"] = pd.to_datetime(revenue_data["Date"])

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=stock_data["Date"],y=stock_data["Close"],name="Stock Price"))

    fig.add_trace(go.Scatter(x=revenue_data["Date"],y=revenue_data["Revenue"],name="Revenue"))

    fig.update_layout(title=f"{stock_name} Stock Price and Revenue Over Time",xaxis_title="Date",yaxis_title="USD",
                      legend_title="Metric",template="plotly_dark",width=800,height=450)

    fig.show()
    
make_graph(gme_data, gme_revenue, "GameStop")
